This notebook provides examples to go along with the [textbook](http://manipulation.csail.mit.edu/robot.html).  I recommend having both windows open, side-by-side!

In [ ]:
from pydrake.all import ModelVisualizer, StartMeshcat, PackageMap, Simulator

from manipulation import running_as_notebook, ConfigureParser, FindResource
from manipulation.station import load_scenario, MakeHardwareStation

In [ ]:
# Start the visualizer.
meshcat = StartMeshcat()

In [ ]:
def AddSpotRemote(parser):
    parser.package_map().AddRemote(
        package_name="spot_description",
        params=PackageMap.RemoteParams(
            urls=[
                f"https://github.com/bdaiinstitute/spot_ros2/archive/d429947a1df842ec38f8c6099dde9501945090d6.tar.gz"
            ],
            sha256=(
                "e4dd471be4e7e822a12afcfd6a94ce7ecbb39e2d4ea406779a96e146a607bf53"
            ),
            strip_prefix="spot_ros2-d429947a1df842ec38f8c6099dde9501945090d6/spot_description/",
        ),
    )

# Simplified Spot model for mobile manipulation

First we'll use the ModelVisualizer to inspect the model.

In [ ]:
visualizer = ModelVisualizer(meshcat=meshcat)
ConfigureParser(visualizer.parser())
AddSpotRemote(visualizer.parser())
visualizer.AddModels(
    url="package://manipulation/spot/spot_with_arm_and_floating_base_actuators.urdf"
)
visualizer.Run(loop_once=not running_as_notebook)
meshcat.DeleteAddedControls()

Now we can use HardwareStation to create a basic simulation.

In [ ]:
scenario = load_scenario(
    filename=FindResource(
        "models/spot/spot_with_arm_and_floating_base_actuators.scenario.yaml"
    )
)
station = MakeHardwareStation(
    scenario, meshcat, parser_preload_callback=AddSpotRemote
)
simulator = Simulator(station)
context = simulator.get_mutable_context()
x0 = station.GetOutputPort("spot.state_estimated").Eval(context)
station.GetInputPort("spot.desired_state").FixValue(context, x0)
simulator.AdvanceTo(0.1);

You can also load the model with cameras.

In [ ]:
scenario = load_scenario(
    filename=FindResource(
        "models/spot/spot_with_arm_and_cameras.scenario.yaml"
    )
)
station = MakeHardwareStation(
    scenario, meshcat, parser_preload_callback=AddSpotRemote
)
simulator = Simulator(station)
context = simulator.get_mutable_context()
x0 = station.GetOutputPort("spot.state_estimated").Eval(context)
station.GetInputPort("spot.desired_state").FixValue(context, x0)
simulator.AdvanceTo(0.1);